<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/biagiominio/Remote_sensing_2021/blob/main/R_code_multivariate_analysis.ipynb#scrollTo=WfzIAI-J6TEO">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/biagiominio/Remote_sensing_2021/blob/main/R_code_multivariate_analysis.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Dimensionality Reduction 
[Principal components analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (PCA) is a technique applied to multispectral and hyperspectral remotely sensed data. PCA transforms an original correlated dataset into a substantially smaller set of uncorrelated variables that represents most of the information present in the original dataset. It reduces data dimensionality (e.g., number of bands). Instead of throwing away the redundant data, principal components analysis condenses the information in intercorrelated variables into a few variables, called principal components.




### Install R packages
The [install.packages](https://www.rdocumentation.org/packages/utils/versions/3.6.2/topics/install.packages) function is used to download and install packages from CRAN-like repositories.
To install multiple packages at the same time, you define a vector that contains the names of the packages to be installed.
The [raster](https://www.rdocumentation.org/packages/raster/versions/3.4-10) package defines classes and methods for spatial raster data access and manipulation. 
[RStoolbox](https://www.rdocumentation.org/packages/RStoolbox/versions/0.2.6) is an package providing a wide range of tools for your every-day remote sensing processing needs. The available tool-set covers many aspects for remote sensing image processing and analysis such as calculating spectral indices, principal component transformation, unsupervised and supervised classification or fractional cover analyses.

In [ ]:
packages <- c("raster", "RStoolbox")
install.packages(packages)

The [library](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/library) function is used to load previously installed packages.
Using the [lapply](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/lapply) function we simultaneously apply the `library()` function to all elements of the vector containing the packages.



In [ ]:
lapply(packages , library, character.only = TRUE)

Set the current working directory of the R process with the [setwd](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/getwd) function.

In [8]:
setwd("/content")

## Study area
Let's analyze Landsat images of the [*Parakanã Indigenous Reserve*](https://pib.socioambiental.org/en/Povo:Parakan%C3%A3) in the state of Pará, a northern region of Brazil. This reserve, founded in 1975, is listed as an indigenous area but has not been assigned to an International Union for Conservation of Nature (IUCN) category (Protected Planet, http://www.protectedplanet.net). Large-scale deforestation in the region is due to agricultural expansion and cattle ranching and has modified this region considerably in the past. Mainly, the Parakanã Indigenous Reserve remains forested. A reservoir on the Rio Tocantins (Tocantins River) covers the north-east part of the area. This large body of water (Lago Tucuruí) is an artificial lake created by the construction of a hydroelectric dam called Represa de Tucuruí (Tucuruí Dam) which was finished in 1984. Construction of the dam is closely related to the observed deforestation and degradation patterns, as workers began settling down after the construction of the dam was complete. Moreover the area is bisected by the Transamazônica highway. Download Landsat images of the Parakanã Indigenous Reserve from this [link]() to get the data to use in this example.

![](https://media.gettyimages.com/photos/large-deforestation-surrounds-the-parakana-indigenous-reserve-in-the-picture-id959485258)


## Reading spatial data
To read and view a multilevel object it is common to use the [brick](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/brick) function.
Select the images already pre-processed for the year 2011. Explore the properties of the `p224r63_2011` object.

Also there may be multiple functions with the same name in multiple packages. The colon operator allows you to specify the specific function you want: `package::functionname`



In [ ]:
p224r63_2011 <- raster::brick("p224r63_2011_masked.grd")
plot(p224r63_2011)

## Multicollinearity


In [ ]:
plot(p224r63_2011$B1_sre, p224r63_2011$B2_sre, col="red", pch=19, cex=2)

In [ ]:
plot(p224r63_2011$B2_sre, p224r63_2011$B1_sre, col="red", pch=19, cex=2)

funzione pairs

In [ ]:
pairs(p224r63_2011)

In [ ]:
# aggregate cells: resampling (ricampionamento)
p224r63_2011res <- aggregate(p224r63_2011, fact=10)

In [ ]:
par(mfrow=c(2,1))
plotRGB(p224r63_2011, r=4, g=3, b=2, stretch="lin")
plotRGB(p224r63_2011res, r=4, g=3, b=2, stretch="lin")

In [ ]:
p224r63_2011res_pca <- rasterPCA(p224r63_2011res)

In [ ]:
summary(p224r63_2011res_pca$model)

In [ ]:
# dev.off()
plotRGB(p224r63_2011res_pca$map, r=1, g=2, b=3, stretch="lin")